In [27]:
import os
import argparse
import torch
from networks.vnet import VNet
#from test_util import  predict_and_center_cut_all_case, test_single_case, calculate_metric_percase
import numpy as np
from tensorflow.keras.utils import to_categorical

import sys
#sys.path.append('./dataloaders')
#from utils import resample_image3D, sitk_onehot_transform, make_out_itk, array_replace, plot_slice_sample

from tqdm import tqdm
import SimpleITK as sitk

# parser = argparse.ArgumentParser()
# parser.add_argument('--root_path', type=str, default='../data/CTM_dataset/unSegmented', help='Folder of Test Set')
# parser.add_argument('--model', type=str,  default='vnet_supervisedonly_dp', help='model_name')
# parser.add_argument('--gpu', type=str,  default='0', help='GPU to use')
# FLAGS = parser.parse_args()
# os.environ['CUDA_VISIBLE_DEVICES'] = FLAGS.gpu
# snapshot_path = "../model/"+FLAGS.model+"/"
# test_save_path = "../model/prediction/"+FLAGS.model+"_post/"


root_path='../../data/gz_dataset/segmented'
model='vnet_supervisedonly_dp'#'VNet_Binary_CTM'
gpu='0'
os.environ['CUDA_VISIBLE_DEVICES'] = gpu
snapshot_path = "../model/"+model+"/"
# test_save_path = "../model/prediction/unSegmented/"+model+"_post/"
test_save_path = "../model/prediction/reconstruct/"+model
#test_save_path = "../model/prediction/"+model+"_post/"
if not os.path.exists(test_save_path):
    os.makedirs(test_save_path)

num_classes = 4#2
output_size =[128, 128, 64]
with open(root_path + '/../test.list', 'r') as f:
    image_list = f.readlines()
image_list = [os.path.join(root_path,item.replace('\n', ''),"CT.nrrd") for item in image_list]

def predict_and_construct(
    net,listt,
    stride_xy, stride_z, patch_size, num_classes=1, device="cuda",
    old_replaced="CT.nrrd"):

    for item in tqdm(listt):
        sample_name = item.split('/')[-2]
        print(sample_name,':')#win系统改为'\\'
        
        # read image
        image_sitk = sitk.ReadImage(item)
        print("type", type(image_sitk))
        seg_sitk = sitk.ReadImage(item.replace(old_replaced, 'Segmentation.seg.nrrd'))#label_onehot_origin_sitk
        label_sitk = sitk.ReadImage(item.replace(old_replaced, 'Segmentation-label.nrrd'))
        label_onehot_sitk = sitk_onehot_transform(label_sitk)
        
        label_name = [
            'bg',
            seg_sitk.GetMetaData('Segment0_Name'),
            seg_sitk.GetMetaData('Segment1_Name'),
            seg_sitk.GetMetaData('Segment2_Name') 
            ]#人工标注的类别顺序
        newspacing = [1.0, 1.0, 1.0]
        oldspacing = np.abs(image_sitk.GetSpacing())
        print("oldspacing",oldspacing)
        imageoldshape = list(sitk.GetArrayFromImage(image_sitk).transpose((2,1,0)).shape)
        print("imageoldshape:",imageoldshape)
        segoldshape = list(sitk.GetArrayFromImage(seg_sitk).transpose((2,1,0)).shape)
        print("segoldshape:",segoldshape)
        labeloldshape = list(sitk.GetArrayFromImage(label_onehot_sitk).transpose((2,1,0,3)).shape)
        image = resample_image3D(image_sitk,newspacing,method='Linear')
        print("labeloldshape:",labeloldshape)
        label_onehot = resample_image3D(label_onehot_sitk,newspacing,method='Nearest')
        
        # get array
        print("label_name:",label_name)
        image = sitk.GetArrayFromImage(image).transpose((2,1,0))#tanspose之后才能与sitk.GetSize匹配
        label_onehot = np.round( sitk.GetArrayFromImage(label_onehot) ).transpose((2,1,0,3))#tanspose之后才能与sitk.GetSize匹配
        label = sitk.GetArrayFromImage(label_sitk).transpose((2,1,0)).astype('uint8')#np.argmax(label_onehot,axis=-1)
        #plot_slice_sample(image,label,np.nonzero(label)[2].max(),fn=False)
        
        # 灰度标准化
        image = (image - np.mean(image)) / np.std(image)
        image = image.astype(np.float32)
        
        ## 调整类别顺序&合并骨头到背景中，注意：是onehot编码
        target_name = ['bg','proximal femur','diastal femur','fragment']#目标类别顺序
        idx = [label_name.index(name) for name in target_name]
        assert len(idx)==4,'one or more classes missed'
        label_onehot = label_onehot[:,:,:,idx]

        '''## bone归入背景类
        bg = label_onehot[:,:,:,[0,1]].sum(axis=-1)[:,:,:,np.newaxis]
        label_onehot = np.concatenate((bg,label_onehot[:,:,:,2:]),axis=-1)
        assert (np.unique(label_onehot) == [0, 1]).all(), "1: pixel class error"
        ## 转化为非onehot编码以便作图'''
        label = np.argmax(label_onehot, axis=-1).astype('uint8')
        
        # cut( center cut)
        tempL = np.nonzero(label)
        minx, maxx = np.min(tempL[0]), np.max(tempL[0])
        miny, maxy = np.min(tempL[1]), np.max(tempL[1])
        minz, maxz = np.min(tempL[2]), np.max(tempL[2])
        w, h, d = label.shape
        px = max(output_size[0] - (maxx-minx+1), 0) // 2
        py = max(output_size[1] - (maxy-miny+1), 0) // 2
        #pz = max(output_size[2] - (maxz-minz+1), 0) // 2
        minx = max(minx - np.random.randint(10, 20) - px, 0)
        maxx = min(maxx + np.random.randint(10, 20) + px, w-1)
        miny = max(miny - np.random.randint(10, 20) - py, 0)
        maxy = min(maxy + np.random.randint(10, 20) + py, h-1)
        image_cut = image[minx:maxx+1, miny:maxy+1, minz:maxz+1]
        label_cut = label[minx:maxx+1, miny:maxy+1, minz:maxz+1]
        label_onehot_cut = label_onehot[minx:maxx+1, miny:maxy+1, minz:maxz+1, :]
        print("cut image.shape:",image.shape, "cut label.shape:",label.shape)
        #plot_slice_sample(image_cut,label_cut,int((maxx-minx)/2),fn=False)
        
        
        # predict
        label_cut_pred, score_cut_pred = test_single_case(net, image_cut, stride_xy, stride_z, patch_size, num_classes=num_classes, device="cuda")
        # get metric
        single_metric = calculate_metric_percase(label_cut_pred, label_cut, num_classes)
        print(single_metric)
        
        # 将cut之后的图像的预测值恢复为原图大小：
        #image = resample_image3D(image,oldspacing,method='Linear')
        
        print("return:",image.shape)
        tmp = np.zeros(image.shape)
        tmp[minx:maxx+1, miny:maxy+1, minz:maxz+1] = label_cut_pred
        label_pred = tmp.astype('uint8')
        #label_pred = 
        
        label_onehot_pred = to_categorical(label_pred)
        offset = seg_sitk.GetMetaData('Segmentation_ReferenceImageExtentOffset')
        offset = [int(fs) for fs in offset.split()]
        print("sitk.GetArrayFromImage(seg_sitk): ",sitk.GetArrayFromImage(seg_sitk).shape)
        shape = sitk.GetArrayFromImage(seg_sitk).transpose(2,1,0).shape
        label_onehot_pred = label_onehot_pred[
            offset[0]:offset[0]+shape[0],
            offset[1]:offset[1]+shape[1],
            offset[2]:offset[2]+shape[2],
            1:
        ].astype('uint8')
        #plot_slice_sample(image,label_pred,np.nonzero(label)[2].max(),fn=False)
        
        #try:
            # generate sitk object
        label_pred_sitk = sitk.GetImageFromArray(label_pred.transpose(2,1,0))
        label_pred_sitk = resample_image3D(label_pred_sitk,oldspacing,imageoldshape,method='Linear')
        label_pred_sitk.CopyInformation(label_sitk)
        label_onehot_pred_sitk = sitk.GetImageFromArray(label_onehot_pred.transpose(2,1,0,3))
        label_onehot_pred_sitk = resample_image3D(label_onehot_pred_sitk,oldspacing,labeloldshape,method='Nearest')
        #test = list(sitk.GetArrayFromImage(label_onehot_pred_sitk).transpose((2,1,0,3)).shape)
        #print("test",test)
        label_onehot_pred_sitk.CopyInformation(seg_sitk)

            # save to nrrd file
        sitk.WriteImage(label_onehot_pred_sitk, item.replace(old_replaced, 'Segmentation-pred.seg.nrrd'))
        sitk.WriteImage(label_pred_sitk, item.replace(old_replaced, 'Segmentation-pred-label.nrrd'))
        #except:
            #print('case %s:generate asd save sitk error, skip this case'%sample_name)

        
def predict_and_construct_all_case(
    epoch_num, 
    patch_size=(128, 128, 64), 
    stride_xy=64, 
    stride_z=32,
    device='cuda'
):
    
    net = VNet(n_channels=1, n_classes=num_classes, normalization='batchnorm', has_dropout=False).to(device)
    save_mode_path = os.path.join(snapshot_path, 'iter_' + str(epoch_num) + '.pth')
    net.load_state_dict(torch.load(save_mode_path))
    print("init weight from {}".format(save_mode_path))
    net.eval()
    
    predict_and_construct(net,image_list, stride_xy, stride_z, patch_size, num_classes=num_classes, device=device)
    

if __name__ == '__main__':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    predict_and_construct_all_case(6001, patch_size=(128, 128, 64), stride_xy=64, stride_z=32, device=device)











  0%|          | 0/1 [00:00<?, ?it/s]

init weight from ../model/vnet_supervisedonly_dp/iter_6001.pth
100201344 :
type <class 'SimpleITK.SimpleITK.Image'>


  0%|          | 0/1 [00:01<?, ?it/s]

oldspacing [0.447266 0.447266 1.25    ]
imageoldshape: [512, 512, 99]


ValueError: axes don't match array

In [1]:
# -*- coding: utf-8 -*-
import os
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
from skimage import measure
import scipy.ndimage as nd
import SimpleITK as sitk
from tensorflow.keras.utils import to_categorical


def recursive_glob(rootdir='.', suffix=''):
    """Performs recursive glob with given suffix and rootdir
        :param rootdir is the root directory
        :param suffix is the suffix to be searched
    """
    return [os.path.join(looproot, filename)
        for looproot, _, filenames in os.walk(rootdir)
        for filename in filenames if filename.endswith(suffix)]

def get_cityscapes_labels():
    return np.array([
        # [  0,   0,   0],
        [128, 64, 128],
        [244, 35, 232],
        [70, 70, 70],
        [102, 102, 156],
        [190, 153, 153],
        [153, 153, 153],
        [250, 170, 30],
        [220, 220, 0],
        [107, 142, 35],
        [152, 251, 152],
        [0, 130, 180],
        [220, 20, 60],
        [255, 0, 0],
        [0, 0, 142],
        [0, 0, 70],
        [0, 60, 100],
        [0, 80, 100],
        [0, 0, 230],
        [119, 11, 32]])

def get_pascal_labels():
    """Load the mapping that associates pascal classes with label colors
    Returns:
        np.ndarray with dimensions (21, 3)
    """
    return np.asarray([[0, 0, 0], [128, 0, 0], [0, 128, 0], [128, 128, 0],
                       [0, 0, 128], [128, 0, 128], [0, 128, 128], [128, 128, 128],
                       [64, 0, 0], [192, 0, 0], [64, 128, 0], [192, 128, 0],
                       [64, 0, 128], [192, 0, 128], [64, 128, 128], [192, 128, 128],
                       [0, 64, 0], [128, 64, 0], [0, 192, 0], [128, 192, 0],
                       [0, 64, 128]])


def encode_segmap(mask):
    """Encode segmentation label images as pascal classes
    Args:
        mask (np.ndarray): raw segmentation label image of dimension
          (M, N, 3), in which the Pascal classes are encoded as colours.
    Returns:
        (np.ndarray): class map with dimensions (M,N), where the value at
        a given location is the integer denoting the class index.
    """
    mask = mask.astype(int)
    label_mask = np.zeros((mask.shape[0], mask.shape[1]), dtype=np.int16)
    for ii, label in enumerate(get_pascal_labels()):
        label_mask[np.where(np.all(mask == label, axis=-1))[:2]] = ii
    label_mask = label_mask.astype(int)
    return label_mask


def decode_seg_map_sequence(label_masks, dataset='pascal'):
    rgb_masks = []
    for label_mask in label_masks:
        rgb_mask = decode_segmap(label_mask, dataset)
        rgb_masks.append(rgb_mask)
    rgb_masks = torch.from_numpy(np.array(rgb_masks).transpose([0, 3, 1, 2]))
    return rgb_masks

def decode_segmap(label_mask, dataset, plot=False):
    """Decode segmentation class labels into a color image
    Args:
        label_mask (np.ndarray): an (M,N) array of integer values denoting
          the class label at each spatial location.
        plot (bool, optional): whether to show the resulting color image
          in a figure.
    Returns:
        (np.ndarray, optional): the resulting decoded color image.
    """
    if dataset == 'pascal':
        n_classes = 21
        label_colours = get_pascal_labels()
    elif dataset == 'cityscapes':
        n_classes = 19
        label_colours = get_cityscapes_labels()
    else:
        raise NotImplementedError

    r = label_mask.copy()
    g = label_mask.copy()
    b = label_mask.copy()
    for ll in range(0, n_classes):
        r[label_mask == ll] = label_colours[ll, 0]
        g[label_mask == ll] = label_colours[ll, 1]
        b[label_mask == ll] = label_colours[ll, 2]
    rgb = np.zeros((label_mask.shape[0], label_mask.shape[1], 3))
    rgb[:, :, 0] = r / 255.0
    rgb[:, :, 1] = g / 255.0
    rgb[:, :, 2] = b / 255.0
    if plot:
        plt.imshow(rgb)
        plt.show()
    else:
        return rgb

def generate_param_report(logfile, param):
    log_file = open(logfile, 'w')
    # for key, val in param.items():
    #     log_file.write(key + ':' + str(val) + '\n')
    log_file.write(str(param))
    log_file.close()

def cross_entropy2d(logit, target, ignore_index=255, weight=None, size_average=True, batch_average=True):
    n, c, h, w = logit.size()
    # logit = logit.permute(0, 2, 3, 1)
    target = target.squeeze(1)
    if weight is None:
        criterion = nn.CrossEntropyLoss(weight=weight, ignore_index=ignore_index, size_average=False)
    else:
        criterion = nn.CrossEntropyLoss(weight=torch.from_numpy(np.array(weight)).float().cuda(), ignore_index=ignore_index, size_average=False)
    loss = criterion(logit, target.long())

    if size_average:
        loss /= (h * w)

    if batch_average:
        loss /= n

    return loss

def lr_poly(base_lr, iter_, max_iter=100, power=0.9):
    return base_lr * ((1 - float(iter_) / max_iter) ** power)


def get_iou(pred, gt, n_classes=21):
    total_iou = 0.0
    for i in range(len(pred)):
        pred_tmp = pred[i]
        gt_tmp = gt[i]

        intersect = [0] * n_classes
        union = [0] * n_classes
        for j in range(n_classes):
            match = (pred_tmp == j) + (gt_tmp == j)

            it = torch.sum(match == 2).item()
            un = torch.sum(match > 0).item()

            intersect[j] += it
            union[j] += un

        iou = []
        for k in range(n_classes):
            if union[k] == 0:
                continue
            iou.append(intersect[k] / union[k])

        img_iou = (sum(iou) / len(iou))
        total_iou += img_iou

    return total_iou

def get_dice(pred, gt):
    total_dice = 0.0
    pred = pred.long()
    gt = gt.long()
    for i in range(len(pred)):
        pred_tmp = pred[i]
        gt_tmp = gt[i]
        dice = 2.0*torch.sum(pred_tmp*gt_tmp).item()/(1.0+torch.sum(pred_tmp**2)+torch.sum(gt_tmp**2)).item()
        print(dice)
        total_dice += dice

    return total_dice

def get_mc_dice(pred, gt, num=2):
    # num is the total number of classes, include the background
    total_dice = np.zeros(num-1)
    pred = pred.long()
    gt = gt.long()
    for i in range(len(pred)):
        for j in range(1, num):
            pred_tmp = (pred[i]==j)
            gt_tmp = (gt[i]==j)
            dice = 2.0*torch.sum(pred_tmp*gt_tmp).item()/(1.0+torch.sum(pred_tmp**2)+torch.sum(gt_tmp**2)).item()
            total_dice[j-1] +=dice
    return total_dice

def post_processing(prediction):
    prediction = nd.binary_fill_holes(prediction)
    label_cc, num_cc = measure.label(prediction,return_num=True)
    total_cc = np.sum(prediction)
    measure.regionprops(label_cc)
    for cc in range(1,num_cc+1):
        single_cc = (label_cc==cc)
        single_vol = np.sum(single_cc)
        if single_vol/total_cc<0.2:
            prediction[single_cc]=0

    return prediction


# +
def resample_image3D(
    image3D,
    newspacing=[1.0,1.0,1.0],
    newsize=None,
    method='Linear',):
    """做插值"""
    resample = sitk.ResampleImageFilter()
    if method == 'Linear':
        resample.SetInterpolator(sitk.sitkLinear)
    elif method == 'Nearest':
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    resample.SetOutputDirection(image3D.GetDirection())
    resample.SetOutputOrigin(image3D.GetOrigin())
    resample.SetOutputSpacing(newspacing)

    if not newsize:
        newsize = np.round(np.array(image3D.GetSize())*np.abs(image3D.GetSpacing())/np.array(newspacing)).astype('int').tolist()
    resample.SetSize(newsize)
    # resample.SetDefaultPixelValue(0)

    newimage = resample.Execute(image3D)
    return newimage

def sitk_onehot_transform(image):
    image_array = sitk.GetArrayFromImage(image)
    label_array_onehot = to_categorical(image_array)
    image_onehot = sitk.GetImageFromArray(label_array_onehot)
    image_onehot.SetOrigin(image.GetOrigin())
    image_onehot.SetDirection(image.GetDirection())
    image_onehot.SetSpacing(image.GetSpacing())
    return image_onehot

def make_out_itk(image,image_sitk):
    out_image_sitk = sitk.GetImageFromArray(image)
    out_image_sitk.SetSpacing(image_sitk.GetSpacing())
    out_image_sitk.SetOrigin(image_sitk.GetOrigin())
    out_image_sitk.SetDirection(image_sitk.GetDirection())
    return out_image_sitk

# 数组替换元素
def array_replace(array,olds,news):
    # 不适用于onehot
    #olds:list of old value
    #news:list of new value
    olds = np.array(olds)
    news = np.array(news)
    offset = olds.max()*10
    tmps = olds+offset
    array += offset
    for tmp,new in zip(tmps,news):
        array[array==tmp] = new
    return array


# -
def plot_slice_sample(image,label,d,fn=False):
    fig = plt.figure()
    a = fig.add_subplot(1, 2, 1)
    imgplot = plt.imshow(image[:,:,d].squeeze())
    a.set_title('image')
    plt.colorbar(orientation='horizontal')
    a = fig.add_subplot(1, 2, 2)
    imgplot = plt.imshow(label[:,:,d].squeeze())
    imgplot.set_clim(0.0, 3.0)
    a.set_title('label')
    plt.colorbar(orientation='horizontal')
    if fn:
        plt.savefig(fn)
    plt.show()



In [2]:
# -*- coding: utf-8 -*-
# +
import h5py
import math
import nibabel as nib
import numpy as np
from medpy import metric
import torch
import torch.nn.functional as F
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os

def predict_and_center_cut_single_case(
    net,image_path,out_dir,num_classes,
    patch_size=(112, 112, 80), stride_xy=18, stride_z=4, 
    save_result=True, test_save_path=None, 
    preproc_fn=None,
    device='cpu'
):
    # 预测并最小包络切割单个样本
    
    if not os.path.isdir(out_dir):
        os.mkdir(out_dir)
        
    h5f = h5py.File(image_path, 'r')
    image = h5f['image'][:]
    if preproc_fn is not None:
        image = preproc_fn(image)
    label_pred, score_map = test_single_case(
        net, image, 
        stride_xy, stride_z, patch_size, 
        num_classes=num_classes, 
        device=device)
        
    # 发现圆形视场的边界处经常出现错误分割(轮廓线),因此需要手动过滤
    r = label_pred.shape[0]/2
    xc,yc = label_pred.shape[0]/2,label_pred.shape[0]/2
    filter_mask = np.ones(label_pred.shape)
    for x in range(label_pred.shape[0]):
        for y in range(label_pred.shape[1]):
            filter_mask[x,y,:] = 0 if r*0.6<np.sqrt((x-xc)**2+(y-yc)**2)<r*2 else 1
    label_pred = filter_mask*label_pred
        
    # 过滤掉小的连通域
    filter_mask = filter_connected_domain(label_pred,min_region_area=100,num_keep_region=None,ratio_keep=None)
    filter_mask = (filter_mask>0).astype(float)
    label_pred = label_pred*filter_mask
        
    # onehot
    label_onehot_pred = tf.keras.utils.to_categorical(label_pred)
    if not label_onehot_pred.shape[-1]==num_classes:
        print(id+' onehot shape error: miss one or more pixel class')
        return None
            
    # center cut
    tempL = np.nonzero(label_pred)
    minx, maxx = np.min(tempL[0]).astype(int), np.max(tempL[0]).astype(int)
    miny, maxy = np.min(tempL[1]).astype(int), np.max(tempL[1]).astype(int)
    minz, maxz = np.min(tempL[2]).astype(int), np.max(tempL[2]).astype(int)
    image = image[minx:maxx+1, miny:maxy+1, minz:maxz+1]
    label_pred = label_pred[minx:maxx+1, miny:maxy+1, minz:maxz+1]
    label_onehot_pred = label_onehot_pred[minx:maxx+1, miny:maxy+1, minz:maxz+1, :]
        
    # image & laberl & pred 拼接
    numd = []
    for d in range(label_pred.shape[2]):
        numd.append( len(np.where(label_pred[:,:,d].flatten()==(num_classes-1))[0]) )
    numd = np.array(numd)
    slice = int(np.where(numd==numd.max())[0][0])
    fig = plt.figure( frameon=False)#dpi=100, 
    image_unstd = (image-image.min())/(image.max()-image.min())*255
    npimg = np.append( image_unstd[:,:,slice],label_pred[:,:,slice]/(num_classes-1)*255,axis=1 )
    plt.imshow(npimg.astype(int),cmap='plasma')#一定要转为int
    plt.savefig(  os.path.join(out_dir, "center_cut_pred.png") )
    plt.show()
    
    return image,label_pred
    
def predict_and_center_cut_all_case(
    net, image_list, num_classes, 
    patch_size=(112, 112, 80), stride_xy=18, stride_z=4, 
    save_result=True, test_save_path=None, 
    preproc_fn=None,
    device='cpu'):
    
    for image_path in tqdm(image_list):
        id = image_path.split('/')[-2]
        print(id,':')
        out_dir = os.path.join(test_save_path ,id) 
        try:
            image,label_pred = predict_and_center_cut_single_case(
                net,
                image_path,
                out_dir,
                num_classes,
                patch_size, stride_xy, stride_z, 
                save_result, test_save_path, 
                preproc_fn,
                device
            )
            if save_result:
                # save files
                filename = os.path.join(out_dir,'center_cut.h5')
                f = h5py.File(filename, 'w')
                f.create_dataset('image', data=image.astype(np.float32), compression="gzip")
    #             f.create_dataset('label', data=label_onehot_pred.astype(np.int), compression="gzip")
                f.close()
    #             nib.save(nib.Nifti1Image(image[:].astype(np.float32), np.eye(4)), 
    #                      out_dir+ '/' + id +'_minx%d_maxx%d_miny%d_maxy%d_minz%d_maxz%d'%(minx,maxx,miny,maxy,minz,maxz)+ "_img.nii.gz")
    #             nib.save(nib.Nifti1Image(label_pred.astype(np.float32), np.eye(4)), 
    #                      out_dir+ '/' + id +'_minx%d_maxx%d_miny%d_maxy%d_minz%d_maxz%d'%(minx,maxx,miny,maxy,minz,maxz)+ "_pred.nii.gz")
    #             nib.save(nib.Nifti1Image(label_onehot_pred[:].astype(np.float32), np.eye(4)), 
    #                      out_dir+ '/' + id +'_minx%d_maxx%d_miny%d_maxy%d_minz%d_maxz%d'%(minx,maxx,miny,maxy,minz,maxz)+ "_label_onehot_pred.nii.gz")
        except:
            print('skip case %s'%id)
    print('All finished')


# -

from skimage import measure
def filter_connected_domain(image,min_region_area=None,num_keep_region=None,ratio_keep=None):
    """
    原文链接：https://blog.csdn.net/a563562675/article/details/107066836
    return label of filter 
    """
    # 标记输入的3D图像
    label, num = measure.label(image, connectivity=1, background=0, return_num=True)
    if num < 1:
        return image

    # 获取对应的region对象
    region = measure.regionprops(label)
    # 获取每一块区域面积并排序
    num_list = [i for i in range(0, num)]
    area_list = [region[i].area for i in num_list]
    
    # 去除面积较小的连通域
    if min_region_area:
        drop_list = np.where(np.array(area_list)<min_region_area)[0]
        for i in drop_list:
            label[region[i].slice][region[i].image] = 0
    elif ratio_keep:
        max_region_area = np.array(area_list).max()
        drop_list = np.where(np.array(area_list)<max_region_area*ratio_keep)[0]
        for i in drop_list:
            label[region[i].slice][region[i].image] = 0 
    
    else:
        if len(num_list) > num_keep_region:
            num_list_sorted = sorted(num_list, key=lambda x: area_list[x])[::-1]# 面积由大到小排序
            for i in num_list_sorted[num_keep_region:]:
                # label[label==i] = 0
                label[region[i].slice][region[i].image] = 0
#             num_list_sorted = num_list_sorted[:num_keep_region]
    return label


def test_all_case(
    net, image_list, 
    num_classes, 
    name_classes,
    patch_size=(112, 112, 80), stride_xy=18, stride_z=4, 
    save_result=True, test_save_path=None, preproc_fn=None,
    device="cuda"
):
    if num_classes==2:
        cols = ['dice','jc','hd','asd']
    else:
        cols = [['dice']*len(name_classes)+['jc']*len(name_classes)+['hd']*len(name_classes)+['asd']*len(name_classes), name_classes*4]
    metrics = pd.DataFrame(columns=cols) 

    for image_path in tqdm(image_list):
        id = image_path.split('/')[-2]
        h5f = h5py.File(image_path, 'r')
        image = h5f['image'][:]
        label = np.argmax(h5f['label'][:],axis=-1)
        if preproc_fn is not None:
            image = preproc_fn(image)
        prediction, score_map = test_single_case(net, image, stride_xy, stride_z, patch_size, num_classes=num_classes, device="cuda")

        if np.sum(prediction)==0:
            single_metric = (0,0,0,0)
        else:
            single_metric = calculate_metric_percase(prediction, label[:], num_classes)
        
        print(id,':')
        print("single_metric:",single_metric)

        metrics.loc[id] = np.array(single_metric).flatten().tolist()
        if save_result:
            nib.save(nib.Nifti1Image(prediction.astype(np.float32), np.eye(4)), test_save_path + id + "_pred.nii.gz")
            nib.save(nib.Nifti1Image(image[:].astype(np.float32), np.eye(4)), test_save_path + id + "_img.nii.gz")
            nib.save(nib.Nifti1Image(label[:].astype(np.float32), np.eye(4)), test_save_path + id + "_gt.nii.gz")
    mean_metrics = metrics.mean()
    print('mean metric is:\n')
    print(mean_metrics)

    return metrics


def test_single_case(net, image, stride_xy, stride_z, patch_size, num_classes=1, device="cuda"):
    w, h, d = image.shape

    # if the size of image is less than patch_size, then padding it
    add_pad = False
    if w < patch_size[0]:
        w_pad = patch_size[0]-w
        add_pad = True
    else:
        w_pad = 0
    if h < patch_size[1]:
        h_pad = patch_size[1]-h
        add_pad = True
    else:
        h_pad = 0
    if d < patch_size[2]:
        d_pad = patch_size[2]-d
        add_pad = True
    else:
        d_pad = 0
    wl_pad, wr_pad = w_pad//2,w_pad-w_pad//2
    hl_pad, hr_pad = h_pad//2,h_pad-h_pad//2
    dl_pad, dr_pad = d_pad//2,d_pad-d_pad//2
    if add_pad:
        image = np.pad(image, [(wl_pad,wr_pad),(hl_pad,hr_pad), (dl_pad, dr_pad)], mode='constant', constant_values=0)
    ww,hh,dd = image.shape

    sx = math.ceil((ww - patch_size[0]) / stride_xy) + 1
    sy = math.ceil((hh - patch_size[1]) / stride_xy) + 1
    sz = math.ceil((dd - patch_size[2]) / stride_z) + 1
    print("{}, {}, {}".format(sx, sy, sz))
    score_map = np.zeros((num_classes, ) + image.shape).astype(np.float32)
    cnt = np.zeros(image.shape).astype(np.float32)

    for x in range(0, sx):
        xs = min(stride_xy*x, ww-patch_size[0])
        for y in range(0, sy):
            ys = min(stride_xy * y,hh-patch_size[1])
            for z in range(0, sz):
                zs = min(stride_z * z, dd-patch_size[2])
                test_patch = image[xs:xs+patch_size[0], ys:ys+patch_size[1], zs:zs+patch_size[2]]
                test_patch = np.expand_dims(np.expand_dims(test_patch,axis=0),axis=0).astype(np.float32)
                test_patch = torch.from_numpy(test_patch).to(device)# cpu
                y1 = net(test_patch)
                y = F.softmax(y1, dim=1)
                y = y.cpu().data.numpy()
                y = y[0,:,:,:,:]
                score_map[:, xs:xs+patch_size[0], ys:ys+patch_size[1], zs:zs+patch_size[2]] += y
                cnt[xs:xs+patch_size[0], ys:ys+patch_size[1], zs:zs+patch_size[2]] += 1
    score_map = score_map/np.expand_dims(cnt,axis=0)
    label_map = np.argmax(score_map, axis = 0)
    if add_pad:
        label_map = label_map[wl_pad:wl_pad+w,hl_pad:hl_pad+h,dl_pad:dl_pad+d]
        score_map = score_map[:,wl_pad:wl_pad+w,hl_pad:hl_pad+h,dl_pad:dl_pad+d]
    return label_map, score_map

def cal_dice(prediction, label, num=2):
    total_dice = np.zeros(num-1)
    for i in range(1, num):
        prediction_tmp = (prediction==i)
        label_tmp = (label==i)
        prediction_tmp = prediction_tmp.astype(np.float)
        label_tmp = label_tmp.astype(np.float)

        dice = 2 * np.sum(prediction_tmp * label_tmp) / (np.sum(prediction_tmp) + np.sum(label_tmp))
        total_dice[i - 1] += dice

    return total_dice

def calculate_metric_percase(pred, gt, num_classes):
    "二分类、多分类的指标统计"
    if num_classes is None:
        num_classes = len(np.unique(gt))#注意：gt不是onehot编码
    print('np.unique(gt):',np.unique(gt))
    if num_classes==2:
        dice = metric.binary.dc(pred, gt)
        jc = metric.binary.jc(pred, gt)
        hd = metric.binary.hd95(pred, gt)
        asd = metric.binary.asd(pred, gt)
    elif num_classes>2:
        from keras.utils import to_categorical
        gt_onehot = to_categorical(gt, num_classes)
        pred_onehot = to_categorical(pred, num_classes)
        dice = []
        jc = []
        hd = []
        asd = []
        for k in range(num_classes):
            pred_k = pred_onehot[...,k]
            gt_k = gt_onehot[...,k]
            dice +=  [metric.dc(result=pred_k, reference=gt_k)]
            jc += [metric.jc(result=pred_k, reference=gt_k)]
            hd += [metric.hd95(result=pred_k, reference=gt_k)]
            asd += [metric.asd(result=pred_k, reference=gt_k)]
    else:
        raise ValueError("pred和gt不能是onehot编码")
    return dice, jc, hd, asd





In [ ]:
'minX%d_maxX%d_minY%d_maxY%d_minZ%d_maxZ%d'%(2,3,4,5,6,7)

In [ ]:
import SimpleITK as sitk

In [ ]:
int('(0,1,2)')